In [ ]:
from flask import Flask, render_template, redirect, url_for

from game_lib import BASIC_ITEMS, Character

In [ ]:
all_items = BASIC_ITEMS

In [ ]:
bob = Character("Bob", 100)
bob.inventory.items = [all_items[0], all_items[1]]

carl = Character("Carl", 100)

lydia = Character("Lydia", 100)
lydia.inventory.items = [all_items[0], all_items[1], all_items[2]]

all_characters = [bob, carl, lydia]

In [ ]:
app = Flask(__name__, template_folder='templates')

In [ ]:
@app.route('/')
def index():
    return redirect(url_for('home'))


@app.route('/home')
def home():
    return render_template('home.html')


@app.route('/items')
def items():
    # TODO create new items
    # TODO remove existing items
    return render_template('items.html', items=all_items)


@app.route('/items/<item_name>')
def item_details(item_name):
    # TODO edit item
    item = next(filter(lambda x: x.name.lower() == item_name, all_items), None)
    return render_template('item_details.html', item=item)


@app.route('/characters')
def characters():
    # TODO create new characters
    # TODO remove existing characters
    return render_template('characters.html', characters=all_characters)


@app.route('/characters/<character_name>')
def character_details(character_name):
    # TODO add or remove gold
    # TODO edit character's name and health
    # TODO explore and change the character's inventory 
    character = next(filter(lambda x: x.name.lower() == character_name, all_characters), None)
    return render_template('character_details.html', character=character)

In [ ]:
app.run()